### Problem 2

Implement GMM clustering on the MNIST Dataset using spherical and diagonal Gaussians

In [1]:
# Loading the required libraries
import numpy as np
import idx2numpy
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from scipy.special import logsumexp
from sklearn.metrics import accuracy_score

In [2]:
# Loading the dataset
test_datapath = './Datasets/Test Images/t10k-images.idx3-ubyte'
train_datapath = './Datasets/Train Images/train-images.idx3-ubyte'
test_labelpath = './Datasets/Test Labels/t10k-labels.idx1-ubyte'
train_labelpath = './Datasets/Train Labels/train-labels.idx1-ubyte'

# Reading the u-byte files and converting them into numpy arrays
X_train = idx2numpy.convert_from_file(train_datapath)
y_train = idx2numpy.convert_from_file(train_labelpath)

X_test = idx2numpy.convert_from_file(test_datapath)
y_test = idx2numpy.convert_from_file(test_labelpath)

# Filtering the dataset to only include the digits 0-4
train_idx = np.where((y_train == 0) | (y_train == 1) | 
                     (y_train == 2) | (y_train == 3) | (y_train == 4))

test_idx = np.where((y_test == 0) | (y_test == 1) | 
                     (y_test == 2) | (y_test == 3) | (y_test == 4))

# Subsetting the dataset
X_train = X_train[train_idx]
X_test = X_test[test_idx]
y_train = y_train[train_idx]
y_test = y_test[test_idx]

# Normalizing the pixel values for easier computation
X_train = X_train/255
X_test = X_test/255


In [3]:
# Compressing the images into 14x14 pixels
X_train_compressed = np.zeros((X_train.shape[0], 14,14))
X_test_compressed = np.zeros((X_test.shape[0], 14,14))

for i in range(X_train.shape[0]):
    X_train_compressed[i] = np.add.reduceat(np.add.reduceat(X_train[i], np.arange(0,X_train[i].shape[0],2), axis=0)
                                            , np.arange(0,X_train[i].shape[1],2), axis=1)/4
    
for i in range(X_test.shape[0]):
    X_test_compressed[i] = np.add.reduceat(np.add.reduceat(X_test[i], np.arange(0,X_test[i].shape[0],2), axis=0)
                                            , np.arange(0,X_test[i].shape[1],2), axis=1)/4

In [4]:
# Reshaping the dataset to be of the form (n, 14*14)
X_train_reshaped = X_train_compressed.reshape(X_train_compressed.shape[0], X_train_compressed.shape[1]*X_train_compressed.shape[2])
X_test_reshaped = X_test_compressed.reshape(X_test_compressed.shape[0], X_test_compressed.shape[1]*X_test_compressed.shape[2])

In [32]:
def generate_data(k, cluster_size, spherical=False, init = True, mean = np.zeros(1), cov = np.zeros(1), seed  = np.random.randint(1,1000)):
    """
    Summary
    A custom function that generates a stack of multivariate normal distributions 

    Args:
        k (int): Dimensions of the distributions to generate
        cluster_size (int): Number of distributions to generate
        spherical (bool, optional): Indicator for Spherical or Diagonal Gaussians. Defaults to False.
        init (bool, optional): Indicator for whether its the first time the functions called or not. Defaults to True.
        mean (list, optional): Mean vector if init = False. Defaults to np.zeros(1).
        cov (list, optional): Cov vector if init = False. Defaults to np.zeros(1).
        seed (int, optional): Random seed generator. Defaults to 42.

    Returns:
        np.array: Stack of multivariate normal distributions
    """
    
    # Initializing the random seed
    np.random.seed(seed)
    
    
    # Check if its the first time the function is called
    if init:
        # Generating the mean vectors
        means = np.array([np.random.rand(k) for _ in range(cluster_size)])
        # Check if the distribution is spherical or diagonal
        if spherical:
            # Generating the covariance matrices  
            covariance_matrix = [np.diag(np.ones(k)*np.random.rand()) for _ in range(cluster_size)]
        else:
            covariance_matrix = [np.diag(np.random.rand(k)) for _ in range(cluster_size)]
        
        # Creating the stack of multivariate normal distributions
        normal_distributions = [multivariate_normal(
            mean=means[i], cov=covariance_matrix[i]) for i in range(cluster_size)]
    else:
        # Creating the stack of multivariate normal distributions based on given values
        normal_distributions = [multivariate_normal(mean = mean[i], cov = cov[i]) 
                                for i in range(cluster_size)]
    
    # Returning the stack of multivariate normal distributions
    return np.array(normal_distributions)
          
        

In [6]:
# Function to calculate Q(theta^(t),theta)
def log_likelihood(X, initial_distributions,pi, clusters):
    """
    Summary
    Function to generate values of Q(theta^(t),theta)
    
    Args:
        X (np.ndarray): Data Matrix
        initial_distributions (list): Stack of multi-variate normal distributions
        pi (list): Probability of each distribution
        clusters (int): Number of clusters

    Returns:
        np.array: Q(theta^(t),theta) values for every cluster
    """
    values = np.array([
        initial_distributions[i].logpdf(X) + np.log(pi[i]) for i in range(clusters)
    ])
    
    return values

In [7]:
def calc_log_likelihood(X, initial_distributions,pi, clusters):
    """Summary
    Calculates the log likelihood of the data given the initial distributions

    Args:
        X (np.ndarray): Data Matrix
        initial_distributions (list): Stack of multi-variate normal distributions
        pi (list): Probability of each distribution
        clusters (int): Number of clusters

    Returns:
        np.float64: Log likelihood of the data given the initial distributions
    """
    values = np.sum([
        initial_distributions[i].pdf(X)* pi[i] for i in range(clusters)
    ])
    
    return np.log(values)

In [8]:
def expectation_step(X,initial_distributions, pi, k):
    """
    Summary

    Function to emulate the expectation step of the EM algorithm
    Args:
        X (np.ndarray): Data Matrix
        initial_distributions (list): Stack of multi-variate normal distributions
        pi (list): Probability of each distribution
        k (int): Number of clusters

    Returns:
        tuple: Stores the value of F and the cluster values for every x_i
    """
    
    # initializing the predictions
    predictions = []
    
    # Calculating Q(theta^(t),theta)
    log_likelihood_values = log_likelihood(X, initial_distributions, pi, k)
    
    # Initializing F 
    F = np.zeros_like(log_likelihood_values.T)
    
    # Parsing the log likelihood values 
    for i in range(log_likelihood_values.T.shape[0]):
        # Calculating the value of F for a data point x_i 
        F[i,] = np.exp(log_likelihood_values[:,i]-logsumexp(log_likelihood_values[:,i]))
        
        # Seeing which cluster the data point belongs to
        max_val = np.argmax(F[i,:])
        
        # Storing the cluster value for the data point
        predictions.append(max_val)
        
    # Returning the value of F and the cluster values for every x_i
    return (F, predictions)

In [9]:
def maximization_step(X, F, clusters):
    """
    Summary
    
    Function to emulate the maximization step of the EM algorithm

    Args:
        X (np.ndarray): Data Matrix
        F (np.ndarray): Value of F for every data point obtained from the expectation step
        clusters (int): Number of clusters

    Returns:
        tuple: vector of means and covariance matrices and updated probabilities for every cluster
    """
    
    # Initializing the means and covariance matrices
    means = []
    covs = []
    pis = []

    # Parsing the F values for a fixed cluster
    for j in range(clusters):
        # Calculating the denominator of the mean and covariance matrix computation 
        denominator = np.sum(F[:,j], axis=0)
        
        # Calculating the mean for the cluster and storing it
        means.append(np.sum(X * F[:,j].reshape(len(X),1), axis=0)/denominator)
        
        # Calculating the covariance diagonal for the cluster and storing it
        cov_diag = []
        for i in range(X.shape[1]):
            # Creating the centering of the expression
            center = ((X[:,j]-means[j][i])**2).reshape(-1,1)
            # Calculating the covariance diagonal for the cluster and storing it
            sigma = (1/denominator) * np.dot(F[:,j].reshape(1, len(X)), center)
            
            # Adding the diagonal to the list and adding 0.05 to prevent underflow
            cov_diag.append(sigma[0][0] + 0.05)
        
        # Storing the covariance diagonal for the cluster
        covs.append(np.diag(cov_diag))
        
        # Calculating the probability of the cluster and storing it
        pis.append(denominator/np.sum(F))
    
    # Returning the vector of means and covariance matrices and updated probabilities for every cluster
    return (means, covs, pis)

In [10]:
def expectation_maximization(X, normal_distributions, pi, k, threshold = 0.0001, maxiters = 1000):
    """
    Summary

    Function to emulate the expectation maximization algorithm 
    
    Args:
        X (np.array): Data Matrix
        normal_distributions (list): Stack of multi-variate normal distributions
        pi (list): Probability of each distribution
        k (int): Number of clusters
        threshold (float, optional): Threshold that reduces the effect of . Defaults to 0.0001.
        maxiters (int, optional): Maximum iterations incase the difference of likelihood doesnt converge. Defaults to 1000.
        
    Returns:
        tuple: vector of predictions, iterations taken and trend of the differences of likelihood
    """
    
    # Initializing the difference of likelihood
    diff = []
    for i in range(maxiters):
        # Calculating the value of log likelihood
        # and the cluster values for every x_i with t^th iteration mean and covariance
        t_1_ll = calc_log_likelihood(X, normal_distributions, pi, k)    
        
        # Running the expectation step
        F,predictions = expectation_step(X, normal_distributions, pi, k)
        # Running the maximization step
        means, covs, pi = maximization_step(X, F, k)
        
        # Updating the normal distributions based on new values of means and covariance matrices 
        normal_distributions = generate_data(X.shape[1], k, mean = means, cov = covs, init = False)
        
        # Calculating the value of log likelihood
        # and the cluster values for every x_i with t+1^th iteration mean and covariance
        t_ll = calc_log_likelihood(X, normal_distributions, pi, k)
        
        # Calculating the difference of likelihood
        t_diff = np.abs(t_ll - t_1_ll)
        
        # Storing the difference of likelihood
        diff.append(t_diff)
        
        # Checking if the difference of likelihood is less than the threshold
        if t_diff < threshold:
            return (predictions,i,diff)
    
    # Returning the vector of predictions, iterations taken and trend of the differences of likelihood
    return (predictions,maxiters,diff)
    

In [11]:
def get_accuracy(y_train,predictions):
    # Subsetting the data based on the predictions
    idx_0 = np.where(np.array(predictions) == 0)
    idx_1 = np.where(np.array(predictions) == 1)
    idx_2 = np.where(np.array(predictions) == 2)
    idx_3 = np.where(np.array(predictions) == 3)
    idx_4 = np.where(np.array(predictions) == 4)

    idxs = [idx_0, idx_1, idx_2, idx_3, idx_4]
    label_map = dict()
    for i,id in enumerate(idxs):
        # Setting label to the cluster
        label_map[i] = np.bincount(y_train[id]).argmax()

    # Generating correct label
    labels = [label_map[i] for i in predictions]

    print("Accuracy: ", accuracy_score(y_train, labels))

In [41]:
# Setting intial parameters
N = X_train.shape[0]
k = 5 
threshold = 0.0001 

# Generating random probabilities adding to one
pi = np.random.dirichlet(np.ones(k))

# Generating the initial distributions for spherical gaussian
spherical_gaussian_distributions_1 = generate_data(X_train_reshaped.shape[1], k, init = True, spherical = True, seed = 1)
spherical_gaussian_distributions_2 = generate_data(X_train_reshaped.shape[1], k, init = True, spherical = True, seed = 2)
spherical_gaussian_distributions_3 = generate_data(X_train_reshaped.shape[1], k, init = True, spherical = True, seed = 3)

# Generating the initial distributions for diagonal gaussian
diagonal_gaussian_distributions_1 = generate_data(X_train_reshaped.shape[1], k, init = True, spherical = False, seed = 1)
diagonal_gaussian_distributions_2 = generate_data(X_train_reshaped.shape[1], k, init = True, spherical = False, seed = 2)
diagonal_gaussian_distributions_3 = generate_data(X_train_reshaped.shape[1], k, init = True, spherical = False, seed = 3)


In [42]:
# Running the expectation maximization algorithm for spherical gaussian
predictions, iterations, diff = expectation_maximization(X_train_reshaped, 
                                                         spherical_gaussian_distributions_1, pi, k, threshold = threshold)

get_accuracy(y_train, predictions)

Accuracy:  0.7105177147339522


In [43]:
# Running the expectation maximization algorithm for spherical gaussian
predictions, iterations, diff = expectation_maximization(X_train_reshaped, 
                                                         spherical_gaussian_distributions_2, pi, k, threshold = threshold)
get_accuracy(y_train, predictions)

Accuracy:  0.7105177147339522


In [44]:
# Running the expectation maximization algorithm for spherical gaussian
predictions, iterations, diff = expectation_maximization(X_train_reshaped, 
                                                         spherical_gaussian_distributions_3, pi, k, threshold = threshold)
get_accuracy(y_train, predictions)

Accuracy:  0.7105177147339522


In [45]:
# Running the expectation maximization algorithm for diagonal gaussian
diagonal_predictions, iterations, diff = expectation_maximization(X_train_reshaped, 
                                                         diagonal_gaussian_distributions_1, pi, k, threshold = threshold)
get_accuracy(y_train, diagonal_predictions)

Accuracy:  0.844260687671591


In [46]:
# Running the expectation maximization algorithm for diagonal gaussian
diagonal_predictions, iterations, diff = expectation_maximization(X_train_reshaped, 
                                                         diagonal_gaussian_distributions_2, pi, k, threshold = threshold)
get_accuracy(y_train, diagonal_predictions)

Accuracy:  0.7106157667668976


In [47]:
# Running the expectation maximization algorithm for diagonal gaussian
diagonal_predictions, iterations, diff = expectation_maximization(X_train_reshaped, 
                                                         diagonal_gaussian_distributions_3, pi, k, threshold = threshold)
get_accuracy(y_train, diagonal_predictions)

Accuracy:  0.8090926918551444
